# Table of Contents
 <p><div class="lev1 toc-item"><a href="#An-Introduction-to-Torch-Framework" data-toc-modified-id="An-Introduction-to-Torch-Framework-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>An Introduction to Torch Framework</a></div><div class="lev2 toc-item"><a href="#Context" data-toc-modified-id="Context-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Context</a></div><div class="lev2 toc-item"><a href="#In-a-nutshell-:" data-toc-modified-id="In-a-nutshell-:-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>In a nutshell :</a></div><div class="lev1 toc-item"><a href="#Implementations" data-toc-modified-id="Implementations-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Implementations</a></div><div class="lev2 toc-item"><a href="#Loading-the-required-modules" data-toc-modified-id="Loading-the-required-modules-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Loading the required modules</a></div><div class="lev2 toc-item"><a href="#Linear-Models---Overview" data-toc-modified-id="Linear-Models---Overview-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Linear Models - Overview</a></div>

# An Introduction to Torch Framework

## Context

* **Institution** Pierre & Marie Curie Sorbonne University
* **Master DAC** Machine Learning and Deep Learning
* **Professors** Patrick Gallinari, Ludovic Denoyer, Nicolas Baskiotis
* **Students** : Youcef Benyettou & David Panou

This notebook follows the instructions given in _"Guide LUA"_ written by **Prof. Denoyer** and **Prof. Baskiotis** at UPMC.
Its intent is to allow the newcomers to get into LUA parametric models.

## In a nutshell : 
1. A loss can :
    * Can calculate its value by comparing the output for a given input 
    * Can compute its derivative for a given entry data point
2. A module can :
    * Can compute it's output for a given input (using its forward method)
    * Can update its gradient using its entry value and is delta (i.e the derivative of its error for a given output which is transmitted by the following module  loss)
    * Can compute its delta (the derivative of its error compared to ..)


# Implementations

## Loading the required modules

In [5]:
require 'nn'
require 'gnuplot'
require 'tools'

In [16]:
-- 1: Create a artificial dataset
DIMENSION=2 -- Entry dataset features dimensions
n_points=1000 -- Number of samples
  
-- We create two gaussians
mean_positive=torch.Tensor(DIMENSION):fill(1); 
var_positive=1.0

mean_negative=torch.Tensor(DIMENSION):fill(-1); 
var_negative=1.0

xs=torch.Tensor(n_points,DIMENSION)
ys=torch.Tensor(n_points,1)
for i=1,n_points/2 do  xs[i]:copy(torch.randn(DIMENSION)*var_positive+mean_positive); ys[i][1]=1 end
for i=n_points/2+1,n_points do xs[i]:copy(torch.randn(DIMENSION)*var_negative+mean_negative); ys[i][1]=-1 end
   

## Linear Models - Overview

A linear model is defined by the following equation :

$$ \hat{Y} = \hat{\beta_{0}} + \sum_{j=1}^{p}X_{j}*\hat{\beta_{j}}$$

With the $\beta$ vector being the model parameters

As they, linear models makes **huge** structural assumption about the data at hand. Thus sometimes, prediction power can be lowered but is very stable.

Such a discussion is about the **bias-variance dilemma** that is reminded at the end of this notebook.

In [39]:
-- 2 : Creation of the model
model = nn.Linear(2,1)
criterion = nn.MSECriterion()
--model:reset(     )

In [ ]:
 -- 3 : Boucle d'apprentissage
 local learning_rate=  
 local maxEpoch=    
 local all_losses={}
 for iteration=1,maxEpoch do
  ------ Mise à jour des paramètres du modèle
      ------ Evaluation de la loss moyenne 
    -- TODO
    local loss=0
    ---- calcul de la loss moyenne 
    all_losses[iteration]=loss  --stockage de la loss moyenne (pour dessin)
  
     -- version gradient stochastique
     -- TODO
  
  -- version gradient batch
  -- TODO
  -- plot de la frontiere ou plot du loss (utiliser l'un ou l'autre)
  --plot(xs,ys,model,100)  -- uniquement si DIMENSION=2
  gnuplot.plot(torch.Tensor(all_losses)) 
end